In [ ]:
pip install transformers datasets torchaudio librosa


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-large-960h-lv60-self")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
!pip install pydub
!apt-get install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import librosa
from pydub import AudioSegment

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")

# Function to extract a short audio clip
def extract_audio_segment(input_path, output_path, start_ms, end_ms):
    # Load the audio file
    audio = AudioSegment.from_file(input_path)
    # Extract the desired segment (start and end in milliseconds)
    segment = audio[start_ms:end_ms]
    # Save the extracted segment to a new file
    segment.export(output_path, format="wav")
    return output_path

# Function to transcribe audio
def transcribe_audio(audio_path):
    # Load and resample the audio to 16kHz
    speech, rate = librosa.load(audio_path, sr=16000)
    input_values = processor(speech, return_tensors="pt", sampling_rate=rate).input_values

    # Perform inference
    with torch.no_grad():
        logits = model(input_values).logits

    # Decode the logits into text
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    return transcription

# Extract a 30-second clip (e.g., first 30 seconds)
short_clip_path = extract_audio_segment(
    input_path="/content/harvard.wav",  # Original file
    output_path="/content/short_clip.wav",  # Output short clip file
    start_ms=0,  # Start time (in ms)
    end_ms=30000  # End time (30 seconds in ms)
)

# Transcribe the short clip
transcription = transcribe_audio(short_clip_path)
print("Short Clip Transcription:", transcription)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Short Clip Transcription: THE STALE SMELL OF OLD BEER LINGERS IT TAKES HEAT TO BRING OUT THE ODOR A COLD DIP RESTORES HEALTH AND ZEST A SALT PICKLE TASTES FINE WITH HAM TAKOS AL PASTOR ARE MY FAVORITE A ZESTFUL FOOD IS THE HOT CROSS BUN


In [ ]:
import librosa

# Function to get audio duration
def get_audio_duration(audio_path):
    # Load the audio file
    speech, rate = librosa.load(audio_path, sr=16000)
    # Calculate duration in seconds
    duration = len(speech) / rate
    return duration

# Calculate duration of the short clip
duration = get_audio_duration("/content/short_clip.wav")
print(f"Duration of audio being converted to text: {duration:.2f} seconds")



Duration of audio being converted to text: 18.36 seconds


In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import librosa
from pydub import AudioSegment

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")

# Function to extract a short audio clip
def extract_audio_segment(input_path, output_path, start_ms, end_ms):
    # Load the audio file
    audio = AudioSegment.from_file(input_path)
    # Extract the desired segment (start and end in milliseconds)
    segment = audio[start_ms:end_ms]
    # Save the extracted segment to a new file
    segment.export(output_path, format="wav")
    return output_path

# Function to transcribe audio
def transcribe_audio(audio_path):
    # Load and resample the audio to 16kHz
    speech, rate = librosa.load(audio_path, sr=16000)
    input_values = processor(speech, return_tensors="pt", sampling_rate=rate).input_values

    # Perform inference
    with torch.no_grad():
        logits = model(input_values).logits

    # Decode the logits into text
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    return transcription

# Extract a 30-second clip (e.g., first 30 seconds)
short_clip_path = extract_audio_segment(
    input_path="/content/Elogios.mp3",  # Original file
    output_path="/content/short2_clip.wav",  # Output short clip file
    start_ms=0,  # Start time (in ms)
    end_ms=30000  # End time (30 seconds in ms)
)

# Transcribe the short clip
transcription = transcribe_audio(short_clip_path)
print("Short Clip Transcription:", transcription)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Short Clip Transcription: ILOCIOS BUENO GOOD GOOD MULIBIEN VERY GOOD VERY GOOD MAGNIFICO WONDERFUL WONDERFUL STUPENDO GREAT GREAT FANTASTICA FANTASTIC FANTASTIC BIENICO WELL


In [ ]:
import librosa

# Function to get audio duration
def get_audio_duration(audio_path):
    # Load the audio file
    speech, rate = librosa.load(audio_path, sr=16000)
    # Calculate duration in seconds
    duration = len(speech) / rate
    return duration

# Calculate duration of the short clip
duration = get_audio_duration("/content/short2_clip.wav")
print(f"Duration of audio being converted to text: {duration:.2f} seconds")


Duration of audio being converted to text: 30.00 seconds


In [1]:
!pip install TTS


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 77.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 112.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 96.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 94.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 60.7 M

In [2]:
from TTS.api import TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)


/usr/local/lib/python3.11/dist-packages/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 | | > y
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|█████████▉| 1.87G/1.87G [00:43<00:00, 42.6MiB/s]
100%|██████████| 1.87G/1.87G [00:44<00:00, 42.2MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 18.9kiB/s]
 58%|█████▊    | 208k/361k [00:00<00:00, 1.67MiB/s]
100%|██████████| 361k/361k [00:00<00:00, 1.01MiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 86.9iB/s]
 97%|█████████▋| 7.51M/7.75M [00:00<00:00, 38.3MiB/s]

 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


100%|██████████| 7.75M/7.75M [00:14<00:00, 38.3MiB/s]/usr/local/lib/python3.11/dist-packages/TTS/tts/layers/xtts/xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featur

In [ ]:
tts.tts_to_file(
    text="Welcome to our project! This application demonstrates advanced voice synthesis and cloning technology, enabling seamless audio replication with natural tones and expressions. Let's explore the power of AI-driven voice technology together!",
    file_path="output.wav",
    speaker_wav="/content/videoplayback",
    language="en"
)


 > Text splitted to sentences.
['Welcome to our project!', 'This application demonstrates advanced voice synthesis and cloning technology, enabling seamless audio replication with natural tones and expressions.', "Let's explore the power of AI-driven voice technology together!"]


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 > Processing time: 14.881837606430054
 > Real-time factor: 0.7172244256464912


'output.wav'

In [ ]:
tts.tts_to_file(
    text="안녕하세요, 제 이름은 정국입니다. 저는 방탄소년단의 멤버입니다. 사랑해요. 저는 당신의 큰 오빠입니다. 당신은 제 동생입니다. 저는 좋은 친구가 있습니다. 제 친구가 그리워요. 곧 만나자, 내 친구. 안녕하세요 김 대통령. 안녕하세요 성 대통령. 만나서 반갑습니다. 저도 만나서 반갑습니다.",
    file_path="output2.wav",
    speaker_wav="/content/harvard.wav",
    language="ko"
)


 > Text splitted to sentences.
['안녕하세요, 제 이름은 정국입니다.', '저는 방탄소년단의 멤버입니다.', '사랑해요.', '저는 당신의 큰 오빠입니다.', '당신은 제 동생입니다.', '저는 좋은 친구가 있습니다.', '제 친구가 그리워요.', '곧 만나자, 내 친구.', '안녕하세요 김 대통령.', '안녕하세요 성 대통령.', '만나서 반갑습니다.', '저도 만나서 반갑습니다.']
 > Processing time: 35.305793046951294
 > Real-time factor: 0.4050464190572221


'output2.wav'

In [9]:
from TTS.api import TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)
tts.tts_to_file(
    text=("Hello, I am an AI bot trying to mimic Elon Musk. "
        "I just came to know he supports Donald Trump. "
        "I didn't expect this, but oh man, he's really smart! "
        "Also democratic. Well, he's more like a cunning, clever person. "
        "Honestly, I think of him as an alien, not a human. "
        "He has bought assets on Mars and thinks people should do the same. "
        "If we want to make Mars like Earth, we need nuclear weapons. "
        "This man is like a supervillain.") ",
    file_path="elon_clone.wav",
    speaker_wav="/content/elon1",
    language="en",
    #pitch=1.2,
    speed=1.1,
)


 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
 > Text splitted to sentences.
['Hello, I am an AI bot trying to mimic Elon Musk.', "I just cam eto know he is a supportor of Doald Trumph and I didn't expect this.", "But ohh mannn, he's really smart, also democratic.", "Well, he's more like a cunning clever person and honestly I think of him as an alien not a human.", 'He has bought assets on Mars and thinks that people sould do the same, if we can to make Mars like our planet Earth then we need to use nuclear weapons on it, as its the fast way.', 'This man is like a super villain.']
 > Processing time: 15.314108848571777
 > Real-time factor: 0.3819588854551341


'elon_clone.wav'

In [12]:
import subprocess

# Define input and output file paths
input_file = "elon_clone.wav"
output_file = "elon_clone_fast2.wav"

# Adjust speed using FFmpeg through subprocess
try:
    subprocess.run([
        "ffmpeg", "-i", input_file, "-filter:a", "atempo=1.25", output_file
    ], check=True)
    print(f"Faster audio generated and saved as {output_file}")
except subprocess.CalledProcessError as e:
    print(f"Error adjusting speed: {e}")



Faster audio generated and saved as elon_clone_fast2.wav
